# Bidirectional LSTM and LSTM

In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import keras
import tensorflow as tf
import warnings
from matplotlib import pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding, Activation, Add
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.regularizers import l2 # L2-regularisation
from keras.models import Model
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

warnings.filterwarnings("ignore")

### Data loader

This notebook has been adapted for Kaggle platform

In [ ]:
def load_data(data_name):
    pkl_file = open(data_name + '.pkl', 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

print('Loading data...')
folder = '../input/neuraldataset2540/neural_'

train = np.array(load_data(folder + 'train'))
train_label = load_data(folder + 'train_label')
test = np.array(load_data(folder + 'test'))
test_label = load_data(folder + 'test_label')
valid = np.array(load_data(folder + 'valid'))
valid_label = load_data(folder + 'valid_label')
print('Data are loaded')

Reshaping for correct input

In [ ]:
train = train.reshape(train.shape[0], train.shape[1], train.shape[2])
test = test.reshape(test.shape[0], test.shape[1], test.shape[2])
valid = valid.reshape(valid.shape[0], valid.shape[1], valid.shape[2])

### BiLSTM

In [ ]:
def bilstm(input_shape, output_shape, mm):
    
    l2_lambda = 0.0001
    
    inputs = Input(input_shape)
    
    layer = (Bidirectional(LSTM(8, return_sequences=True)))(inputs)
    layer = (Bidirectional(LSTM(16)))(layer)

    output = (Dense(mm, activation = 'linear', W_regularizer = l2(l2_lambda)))(layer) 
    output = (LeakyReLU())(output)
    output = (Dropout(0.3))(output)
    
    finish = []
    for i in range(int((mm ** 2 - mm ) / 2)):
        output = (Dense(2, activation = 'softmax', name = 'main_output' + str(i)))(output)
        finish.append(output)

    model = Model(inputs = [inputs], outputs = finish)
    
    model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy'
    )
    
    return model

### LSTM

In [ ]:
def lstm(input_shape, output_shape, mm):
    
    l2_lambda = 0.01
    
    inputs = Input(input_shape)
    
    layer = (LSTM(32, return_sequences=True))(inputs)
    layer = (LSTM(64, return_sequences=True))(layer)
    layer = (LSTM(128))(layer)
        
    layer = (BatchNormalization())(layer)
    output = (Dense(128, activation = 'linear', W_regularizer = l2(l2_lambda)))(layer) 
    output = (ReLU())(layer)
    output = (Dropout(0.3))(output)
    
    finish = []
    for i in range(int((mm ** 2 - mm ) / 2)):
        output = (Dense(2, activation = 'softmax', name = 'main_output' + str(i)))(output)
        finish.append(output)

    model = Model(inputs = [inputs], outputs = finish)
    
    model.compile(
        optimizer = 'adam',
        loss = 'categorical_crossentropy'
    )
    
    return model

Training depending on the model (lstm or bilstm)

In [ ]:
batch_size = 16
epochs = 20
mm = 40

model = lstm((train.shape[1], train.shape[2]), 2, mm)

history = model.fit(train, list(train_label), batch_size = batch_size, epochs = epochs, verbose = 0,
                          validation_data=(valid, list(valid_label)))

Predictions

In [ ]:
predicted = model.predict(test)

print(np.shape(predicted))
print(np.shape(test_label))

output = open('predicted_bi' + '.pkl', 'wb')
pickle.dump(predicted, output)
output.close()

output = open('test_bi' + '.pkl', 'wb')
pickle.dump(test_label, output)
output.close()

Plot the loss

In [ ]:
# Plot training & validation loss values

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('bilstm')